# Pre-forecast demand for faster training

In [49]:
# imports
import pandas as pd
import numpy as np
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))

from demand_forecasting.IrConv_LSTM_demand_forecaster import IrConvLstmDemandForecaster

In [62]:
# constants
DROPOFF_MODEL_PATH = "/home/ruroit00/rebalancing_framework/rl_framework/demand_forecasting/models/irregular_convolution_LSTM_dropoff.pkl"
PICKUP_MODEL_PATH = "/home/ruroit00/rebalancing_framework/rl_framework/demand_forecasting/models/irregular_convolution_LSTM_pickup.pkl"

ZONE_COMMUNITY_MAP_PATH = "/home/ruroit00/rebalancing_framework/processed_data/grid_community_map.pickle"

DROP_OFF_DEMAND_DATA_PATH = '/home/ruroit00/rebalancing_framework/processed_data/voi_dropoff_demand_h3_hourly.pickle'
PICK_UP_DEMAND_DATA_PATH = '/home/ruroit00/rebalancing_framework/processed_data/voi_pickup_demand_h3_hourly.pickle'

DROPOFF_OUTPUT_PATH = '../data/IrConv_LSTM_dropoff_forecasts.pkl'
PICKUP_OUTPUT_PATH = '../data/IrConv_LSTM_pickup_forecasts.pkl'

In [51]:
ZONE_COMMUNITY_MAP: pd.DataFrame = pd.read_pickle(
        ZONE_COMMUNITY_MAP_PATH
    )

N_COMMUNITIES = ZONE_COMMUNITY_MAP["community_index"].nunique()
N_ZONES = ZONE_COMMUNITY_MAP.shape[0]

In [52]:
# load models
dropoff_demand_forecaster = IrConvLstmDemandForecaster(
    num_communities=N_COMMUNITIES,
    num_zones=N_ZONES,
    zone_community_map=ZONE_COMMUNITY_MAP,
    model_path=DROPOFF_MODEL_PATH,
    demand_data_path=DROP_OFF_DEMAND_DATA_PATH,
)

pickup_demand_forecaster = IrConvLstmDemandForecaster(
    num_communities=N_COMMUNITIES,
    num_zones=N_ZONES,
    zone_community_map=ZONE_COMMUNITY_MAP,
    model_path=PICKUP_MODEL_PATH,
    demand_data_path=PICK_UP_DEMAND_DATA_PATH,
)

In [53]:
dropoff_demand_forecaster.scaled_demand_data

array([[[0.       ],
        [0.       ],
        [0.       ],
        ...,
        [0.       ],
        [0.0106383],
        [0.       ]],

       [[0.       ],
        [0.       ],
        [0.       ],
        ...,
        [0.       ],
        [0.0106383],
        [0.       ]],

       [[0.       ],
        [0.       ],
        [0.       ],
        ...,
        [0.       ],
        [0.       ],
        [0.       ]],

       ...,

       [[0.       ],
        [0.       ],
        [0.       ],
        ...,
        [0.       ],
        [0.       ],
        [0.       ]],

       [[0.       ],
        [0.       ],
        [0.       ],
        ...,
        [0.       ],
        [0.       ],
        [0.       ]],

       [[0.       ],
        [0.       ],
        [0.       ],
        ...,
        [0.       ],
        [0.       ],
        [0.       ]]])

In [54]:
dropoff_forecast_data = pd.DataFrame(
    index=dropoff_demand_forecaster.demand_data.index,
    columns=dropoff_demand_forecaster.demand_data.columns,
    data=np.zeros(dropoff_demand_forecaster.demand_data.shape),
)
pickup_forecast_data = pd.DataFrame(
    index=pickup_demand_forecaster.demand_data.index,
    columns=pickup_demand_forecaster.demand_data.columns,
    data=np.zeros(pickup_demand_forecaster.demand_data.shape),
)

In [55]:
# generate dropoff forecasts
for i, row in dropoff_demand_forecaster.demand_data.iterrows():
    try:
        forecast = dropoff_demand_forecaster.predict_demand_per_zone(
            i.hour, i.day, i.month
        )
    except Exception as e:
        print(f"Error predicting dropoff demand for index {i}: {e}")
        forecast = np.zeros(N_ZONES, dtype=float)
    dropoff_forecast_data.loc[i] = forecast[:, 0]

In [56]:
dropoff_forecast_data.describe()

grid_index,881faa4485fffff,881faa4487fffff,881faa4493fffff,881faa4497fffff,881faa44b1fffff,881faa44b3fffff,881faa44b7fffff,881faa44b9fffff,881faa44bbfffff,881faa44bdfffff,...,881faa7ad9fffff,881faa7adbfffff,881faa7addfffff,881faa7ae1fffff,881faa7ae3fffff,881faa7ae5fffff,881faa7ae7fffff,881faa7ae9fffff,881faa7aebfffff,881faa7aedfffff
count,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000,...,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000
mean,0.044914,-0.290660,-0.119890,0.031428,0.164776,-0.020343,0.016094,-0.065017,0.141468,0.177546,...,1.203552,1.084778,1.322676,0.949727,4.079474,-0.111075,0.208152,0.449100,1.917489,0.682703
std,0.046223,0.130576,0.072994,0.082706,0.165819,0.049904,0.059255,0.148414,0.124655,0.072432,...,0.263982,0.116427,0.301702,0.292678,0.901602,0.058060,0.141919,0.150732,0.541154,0.192676
min,-0.074689,-0.824245,-0.319807,-0.489602,-0.814581,-0.239245,-0.244856,-0.748413,-0.523160,-0.146551,...,0.688083,0.910834,0.770699,0.447177,2.554276,-0.216533,-0.319978,0.159338,0.773705,-0.184210
25%,0.012212,-0.375224,-0.178454,-0.020411,0.062228,-0.050389,-0.022698,-0.156543,0.060502,0.130238,...,0.994062,0.990435,1.089911,0.728522,3.388926,-0.155533,0.116001,0.325088,1.447611,0.589214
50%,0.042153,-0.300038,-0.127705,0.023705,0.155845,-0.023452,0.018475,-0.058423,0.150363,0.187782,...,1.166128,1.060405,1.289338,0.916109,3.961305,-0.116629,0.218902,0.417390,1.886356,0.706152
75%,0.075627,-0.204628,-0.072717,0.080257,0.257568,0.009103,0.060929,0.029737,0.224026,0.235803,...,1.398797,1.159011,1.546622,1.138625,4.716725,-0.069987,0.307530,0.548796,2.267844,0.787411
max,0.210155,0.168021,0.141173,0.388880,0.843780,0.154675,0.224395,0.797463,0.582545,0.357833,...,1.948509,1.589939,2.101285,1.778372,6.620086,0.042756,0.789172,1.067223,4.336843,1.961904


In [57]:
# generate pickup forecasts
for i, row in pickup_demand_forecaster.demand_data.iterrows():
    try:
        forecast = pickup_demand_forecaster.predict_demand_per_zone(
            i.hour, i.day, i.month
        )
    except Exception as e:
        print(f"Error predicting dropoff demand for index {i}: {e}")
        forecast = np.zeros(N_ZONES, dtype=float)
    dropoff_forecast_data.loc[i] = forecast[:, 0]

In [58]:
pickup_forecast_data.describe()

grid_index,881faa4485fffff,881faa4487fffff,881faa4493fffff,881faa4497fffff,881faa44b1fffff,881faa44b3fffff,881faa44b7fffff,881faa44b9fffff,881faa44bbfffff,881faa44bdfffff,...,881faa7ad9fffff,881faa7adbfffff,881faa7addfffff,881faa7ae1fffff,881faa7ae3fffff,881faa7ae5fffff,881faa7ae7fffff,881faa7ae9fffff,881faa7aebfffff,881faa7aedfffff
count,1820.0,1820.0,1820.0,1820.0,1820.0,1820.0,1820.0,1820.0,1820.0,1820.0,...,1820.0,1820.0,1820.0,1820.0,1820.0,1820.0,1820.0,1820.0,1820.0,1820.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
# save forecasts
dropoff_forecast_data.to_pickle(DROPOFF_OUTPUT_PATH)
pickup_forecast_data.to_pickle(PICKUP_OUTPUT_PATH)